In [1]:


import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
import lightgbm as lgb
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
#from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
import lightgbm as lgb



In [2]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [3]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [4]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [5]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [ ]:
# scalar=MinMaxScaler()
# X = scalar.fit_transform(X)
# X_testdata = scalar.transform(X_testdata)




In [7]:
# import lightgbm as lgb
# import pandas as pd
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import train_test_split


# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Initialize and train the LightGBM model on full feature set to get feature importances
# clf = lgb.LGBMClassifier(
#     objective='binary',
#     metric='binary_logloss',
#     boosting_type='gbdt',
#     num_leaves=20,
#     max_depth=4,
#     learning_rate=0.03,
#     n_estimators=376,
#     subsample=0.7,
#     colsample_bytree=0.5,
#     min_child_weight=5,
#     scale_pos_weight=1.1,
#     reg_lambda=0.1,
#     verbose=-1
# )

# # Fit the model on full data to get feature importances
# clf.fit(X_train, y_train)

# # Get feature importances and create a DataFrame
# feature_importances = clf.feature_importances_
# feature_importances_df = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': feature_importances
# })

# # Sort the DataFrame by importance
# feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# # Define different numbers of top features to test
# feature_counts = [10, 15, 20, 25, 30,35,40,45,50,55,60]
# auc_scores = {}

# # Iterate over different numbers of top features
# for n in feature_counts:
#     # Select the top n features
#     top_features = feature_importances_df['Feature'].head(n).tolist()

#     # Subset the training and testing data to the top n features
#     X_train_top = X_train[top_features]
#     X_test_top = X_test[top_features]

#     # Re-train the model with the top n features
#     clf.fit(X_train_top, y_train)

#     # Predict probabilities on the test set
#     y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

#     # Calculate AUC score
#     auc = roc_auc_score(y_test, y_pred_proba)
#     auc_scores[n] = auc  # Store the AUC score for this number of features

#     print(f"Top {n} features - AUC: {auc:.4f}")

# # Print all AUC scores for comparison
# print("\nAUC scores for different feature counts:")
# for n, auc in auc_scores.items():
#     print(f"Top {n} features: AUC = {auc:.4f}")


Top 10 features - AUC: 0.9447
Top 15 features - AUC: 0.9555
Top 20 features - AUC: 0.9579
Top 25 features - AUC: 0.9611
Top 30 features - AUC: 0.9632
Top 35 features - AUC: 0.9625
Top 40 features - AUC: 0.9634
Top 45 features - AUC: 0.9629
Top 50 features - AUC: 0.9631
Top 55 features - AUC: 0.9641
Top 60 features - AUC: 0.9644

AUC scores for different feature counts:
Top 10 features: AUC = 0.9447
Top 15 features: AUC = 0.9555
Top 20 features: AUC = 0.9579
Top 25 features: AUC = 0.9611
Top 30 features: AUC = 0.9632
Top 35 features: AUC = 0.9625
Top 40 features: AUC = 0.9634
Top 45 features: AUC = 0.9629
Top 50 features: AUC = 0.9631
Top 55 features: AUC = 0.9641
Top 60 features: AUC = 0.9644


In [12]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:


# Initialize the LightGBM model
clf = lgb.LGBMClassifier(  
    objective= 'binary',
    metric= 'binary_logloss',
    boosting_type= 'gbdt',
    num_leaves= 100,
    max_depth= 8,
    learning_rate= 0.03,
    n_estimators= 1500,
    
    min_data_in_leaf=50,
    
    verbose= -1)

clf.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.03, max_depth=8, metric='binary_logloss',
               min_data_in_leaf=50, n_estimators=1500, num_leaves=100,
               objective='binary', verbose=-1)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

# Initialize the LightGBM model
lgbm_clf = lgb.LGBMClassifier(
    objective='binary',
    metric='binary_logloss',
    boosting_type='gbdt',
    num_leaves=100,
    max_depth=8,
    learning_rate=0.03,
    n_estimators=1500,
    min_data_in_leaf=50,
    verbose=-1
)

# Initialize the Bagging classifier with the LightGBM model as the base estimator
bagging_clf = BaggingClassifier(
    base_estimator=lgbm_clf,
    n_estimators=10,  # Number of base estimators in the ensemble
    max_samples=0.8,  # Fraction of samples to draw from X to train each base estimator
    max_features=0.8,  # Fraction of features to draw from X to train each base estimator
    bootstrap=True,  # Whether samples are drawn with replacement
    bootstrap_features=False,  # Whether features are drawn with replacement
    n_jobs=-1,  # Number of jobs to run in parallel (-1 means using all processors)
    random_state=42  # Random seed for reproducibility
)

# Fit the Bagging classifier to the training data
bagging_clf.fit(X_train, y_train)

# Predict probabilities for the test data
y_pred_proba = bagging_clf.predict_proba(X_test)[:, 1]

# Calculate the AUC-ROC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC Score: {auc_score:.4f}")

c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(feature_importances_df)

top_features = feature_importances_df['Feature'].head(40).tolist()




   Feature  Importance
67     X69        5447
30     X32        4392
64     X66        4059
68     X70        3624
0       X2        3111
..     ...         ...
17     X19         106
74     X76          86
2       X4          30
14     X16           5
69     X71           3

[77 rows x 2 columns]


In [ ]:
X_train_top = X_train[top_features]
X_test_top = X_test[top_features]
X_testdata_top = X_testdata[top_features]

# Re-train the model with the top features
clf.fit(X_train_top, y_train)

LGBMClassifier(learning_rate=0.03, max_depth=8, metric='binary_logloss',
               min_data_in_leaf=50, n_estimators=1500, num_leaves=100,
               objective='binary', verbose=-1)

In [11]:
y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")

AUC: 0.9260563177249185


In [17]:
# Predict probabilities for the test set
y_pred_proba_testdata = clf.predict_proba(X_testdata_top)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[2.28597395e-03 3.28565037e-04 8.64616620e-05 ... 1.97305216e-01
 1.10920409e-04 3.35045108e-05]
        RecordId  Y_probability
0         300001       0.002286
1         300002       0.000329
2         300003       0.000086
3         300004       0.000540
4         300005       0.000421
...          ...            ...
105477    405478       0.000056
105478    405479       0.197305
105479    405480       0.000111
105480    405481       0.000034
105481    405482       0.000232

[105482 rows x 2 columns]
